## AMP® - Parkinson's Disease Progression Prediction - Baseline

## 1. Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd

dataset_directory = Path('/kaggle/input/amp-parkinsons-disease-progression-prediction')
sys.path.append(dataset_directory)
import amp_pd_peptide

In [ ]:
df_train_clinical_data = pd.read_csv(dataset_directory / 'train_clinical_data.csv')
print(f'Train Clinical Data Shape: {df_train_clinical_data.shape} - Memory Usage: {df_train_clinical_data.memory_usage().sum() / 1024 ** 2:.2f} MB')

patient_count = df_train_clinical_data['patient_id'].nunique()
visit_count_mean = df_train_clinical_data.groupby('patient_id')['visit_month'].count().mean()
print(f'Patient Count: {patient_count} - Mean Visit Count: {visit_count_mean:.2f}')

In [ ]:
df_supplemental_clinical_data = pd.read_csv(dataset_directory / 'supplemental_clinical_data.csv')
print(f'Supplemental Clinical Data Shape: {df_supplemental_clinical_data.shape} - Memory Usage: {df_supplemental_clinical_data.memory_usage().sum() / 1024 ** 2:.2f} MB')

patient_count = df_supplemental_clinical_data['patient_id'].nunique()
visit_count_mean = df_supplemental_clinical_data.groupby('patient_id')['visit_month'].count().mean()
print(f'Patient Count: {patient_count} - Mean Visit Count: {visit_count_mean:.2f}')

## 2. Evaluation

The problem is predicting `updrs_1`, `updrs_2`, `updrs_3` and `updrs_4` values at a given `visit_month` and potential **6**, **12** and **24** month visits after the current `visit_month`.

Predictions are evaluated on symmetric mean absolute percentage error (SMAPE) + 1.

In [ ]:
def symmetric_mean_absolute_percentage_error(y_true, y_pred):

    """
    Calculate symmetric mean absolute percentage error from given ground-truth and predictions
    
    Parameters
    ----------
    y_true: array-like of shape (n_samples)
        Array of ground-truth values
        
    y_pred: array-like of shape (n_samples)
        Array of prediction values
        
    Returns
    -------
    smape: float
        Symmetric mean absolute percentage error
    """

    smape = 100 / len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

    return smape


def score(df, target_columns, prediction_columns):
    
    """
    Concatenate targets and prediction into a single array and calculate SMAPE + 1
    
    Parameters
    ----------
    target_columns: list of shape (4)
        Array of target column names
        
    prediction_columns: list of shape (4)
        Array of prediction column names
        
    Returns
    -------
    score: float
        Symmetric mean absolute percentage error
    """
    
    y_true = []
    y_pred = []
    
    for target_column, prediction_column in zip(target_columns, prediction_columns):
        target_idx = df[target_column].notna()
        y_true.append(df.loc[target_idx, target_column].values + 1)
        y_pred.append(df.loc[target_idx, prediction_column].values + 1)
        
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
        
    score = symmetric_mean_absolute_percentage_error(
        y_true=y_true,
        y_pred=y_pred
    )
    
    return score


## 3. Baselines

Median values at each visit month is a strong baseline since this kind of forecasting problems are hard. Visit month median values of `updrs_1` are calculated on clinical data and median values of `updrs_2`, `updrs_3`, `updrs_4` are calculated on clinical and supplemental data.

When clinical and supplemental data are concatenated, different `visit_month` values (5) exist on the baseline table. Unseen `visit_month` values also exist in hidden test set which will be handled accordingly.

Default median values of visit months have a flaw. Median updrs values might decrease as `visit_month` increases since dataset is not large enough. A baseline like that might overfit to training set and generalize poorly. Replacing median values with expanding window max values increases validation and leaderboard scores.

In [ ]:
target_columns_clinical_data = ['updrs_1']
target_columns_clinical_and_supplemental_data = ['updrs_2', 'updrs_3', 'updrs_4']

target_visit_month_medians_clinical_data = df_train_clinical_data.groupby('visit_month')[target_columns_clinical_data].median()
target_visit_month_medians_clinical_and_supplemental_data = pd.concat((
    df_train_clinical_data,
    df_supplemental_clinical_data
), axis=0).groupby('visit_month')[target_columns_clinical_and_supplemental_data].median()

# Drop 5th month visit that is coming from the supplemental clinical data
target_visit_month_medians_clinical_and_supplemental_data = target_visit_month_medians_clinical_and_supplemental_data.drop(5)

# Concatenate visit_month medians of targets
target_visit_month_medians = pd.concat((
    target_visit_month_medians_clinical_data,
    target_visit_month_medians_clinical_and_supplemental_data
), axis=1, ignore_index=False)

# Replace expanding window max of updrs values with current updrs values
target_visit_month_medians = target_visit_month_medians.expanding(min_periods=1).max()
target_visit_month_medians

## 4. Validation

* Medians of `visit_month` groups calculated on train clinical data
  * `updrs_1` OOF: **55.63**
  * `updrs_2` OOF: **70.51**
  * `updrs_3` OOF: **70.94**
  * `updrs_4` OOF: **48.00**
  * Global OOF: **62.71**
  * Public LB Score: **57.8** (filled with global target medians)
  
* Medians of `visit_month` groups calculated on train + supplemental clinical data
  * `updrs_1` OOF: **56.16**
  * `updrs_2` OOF: **70.53**
  * `updrs_3` OOF: **70.16**
  * `updrs_4` OOF: **48.00**
  * Global OOF: **62.65**
  * Public LB Score: **57.2** (filled with global target medians)
  
* Medians of `visit_month` groups calculated on train + supplemental clinical data replaced with expanding window max
  * `updrs_1` OOF: **56.29**
  * `updrs_2` OOF: **70.79**
  * `updrs_3` OOF: **69.19**
  * `updrs_4` OOF: **48.01**
  * Global OOF: **62.49**
  * Public LB Score: **56.4** (filled with global target medians)
  
* Medians of `visit_month` groups calculated on train for `updrs_1` and train + supplemental clinical data for `updrs_2`, `updrs_3`, `updrs_4` 
  * `updrs_1` OOF: **55.74**
  * `updrs_2` OOF: **70.79**
  * `updrs_3` OOF: **69.19**
  * `updrs_4` OOF: **48.01**
  * Global OOF: **62.34**
  * Public LB Score: **56.3** (filled with closest `visit_month` median)

In [ ]:
fold_columns = ['fold1', 'fold2', 'fold3', 'fold4', 'fold5']
target_columns = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']

for target_column in target_columns:
        
    target_idx = df_train_clinical_data[target_column].notna()
    df_train = df_train_clinical_data.loc[target_idx]
    print(f'Target: {target_column} Dataset Shape: {df_train.shape}')
        
    df_train_clinical_data.loc[target_idx, f'{target_column}_predictions'] = df_train_clinical_data.loc[target_idx, 'visit_month'].map(target_visit_month_medians[target_column])
    val_score = score(
        df=df_train_clinical_data.loc[target_idx],
        target_columns=[target_column],
        prediction_columns=[f'{target_column}_predictions']
    )
    print(f'Validation SMAPE: {val_score:.4f}\n')
    
global_oof_score = score(
    df=df_train_clinical_data,
    target_columns=target_columns,
    prediction_columns=[f'{target_column}_predictions' for target_column in target_columns]
)
print(f'Global OOF SMAPE: {global_oof_score:.4f}')

## 5. Submission

Since hidden test has unseen `visit_month` values, there will be some missing predictions after mapping the baselines. In order to deal with those cases, closest `visit_month` baseline values are used for filling missing predictions after the map operation.

In [ ]:
env = amp_pd_peptide.make_env()
test_iterator = env.iter_test() 

for (df_test, df_test_peptides, df_test_proteins, df_submission) in test_iterator:
    
    df_submission['patient_id'] = df_submission.apply('prediction_id').str.split('_', expand=True)[0].astype(int)
    df_submission['current_visit_month'] = df_submission.apply('prediction_id').str.split('_', expand=True)[1].astype(int)
    df_submission['visit_month_offset'] = df_submission.apply('prediction_id').str.split('_', expand=True)[5].astype(int)
    df_submission['prediction_visit_month'] = df_submission['current_visit_month'] + df_submission['visit_month_offset'].astype(int)
    df_submission['updrs'] = df_submission.apply('prediction_id').str.split('_', expand=True)[3].astype(int)

    for updrs in range(1, 5):
        updrs_idx = df_submission['updrs'] == updrs
        df_submission.loc[updrs_idx, 'rating'] = df_submission.loc[updrs_idx, 'prediction_visit_month'].map(target_visit_month_medians[f'updrs_{updrs}'])
        
        missing_idx = df_submission['rating'].isnull()
        # Iterate over missing prediction rows after mapping the baselines
        for idx, row in df_submission[updrs_idx & missing_idx].iterrows():
            # Find the closest visit_month value from the baselines table
            target_visit_month_median_idx = np.argmin(np.abs(target_visit_month_medians.index - row['prediction_visit_month']))
            # Write the closest visit_month value to the unseen visit_month
            df_submission.loc[idx, 'rating'] = target_visit_month_medians.iloc[target_visit_month_median_idx, updrs - 1]
    
    df_submission = df_submission.loc[:, ['prediction_id', 'rating']]
    env.predict(df_submission)
